# Scope Rules

* Most C++ style languages supports scope rules like
    * public
        * accessible everywhere
    * private
        * accessible only within classes
    * protected
        * accessible by class and subclass
    * internal/pacakge
        * accessible within the modules

### Python DOESN'T HAVE SCOPE RULES!

* All memembers are always **public**
* They can be modified from anywhere in the code.

## Python's scope convention

* because python doesn't supports scopes, we do it using convention
* any class member that begins with an underscore is considered private
    * they don't really become private
    * we are expected not to access them directly.
        * there implementation may change tomorrow
        * external codes are not supposed to access them

In [1]:
class BankAccount:
    def __init__(self, accountNumber, name, password, balance, interestRate):
        #it is ok to access it from outside
        self.accountNumber = accountNumber 
        self.name=name 
    
        # please don't directly  to access the below members
        self._password = password
        self._balance = balance
        self._interestRate = interestRate
        

In [2]:
a1=BankAccount(1,'Vivek','p@ss',20000,12)

In [3]:
a1.name # it is ok to access

'Vivek'

In [4]:
a1._password #we shouldn't directly access. but it can be accessed

'p@ss'

### We may provide getter/setter to access those members

In [6]:
class BankAccount:
    def __init__(self, accountNumber, name, password, balance, interestRate):
        #it is ok to access it from outside
        self.accountNumber = accountNumber 
        self.name=name 
    
        # please don't directly  to access the below members
        self._password = password
        self._balance = balance
        self._interestRate = interestRate

    def get_balance(self): 
        return self._balance
    
    def get_interest_rate(self): return self._interestRate

    def set_interest_rate(self, newRate): 
        delta= self._interestRate/10
        if newRate>= self._interestRate - delta and newRate<= self._interestRate+delta:
            self._interestRate = newRate
    
    def authenticate(self, password):
        return self._password==password
    
    def change_password(self, oldPassword, newPassword):
        if self.authenticate(oldPassword):
            self._password=newPassword

In [7]:
a=BankAccount(1,'Vivek','p@ss',20000,12)



In [8]:
a.authenticate("Hi")

False

In [9]:
a.authenticate("p@ss")

True

In [10]:
a.change_password("pass","pass2")

In [11]:
a.authenticate("pass2")

False

In [12]:
a._password # shouldn't access

'p@ss'

In [13]:
a.set_interest_rate(15) # should fail

a.get_interest_rate()

12

In [14]:
a.set_interest_rate(13) # should work
a.get_interest_rate()

13

## A **more** private convention

* In python, we can make a **more priavate** convention
* we can use double underscore prefix "**\_\_**" They are more difficult to break

In [18]:
class BankAccount:
    def __init__(self, accountNumber, password, balance):
        self._accountNumber=accountNumber
        self.__password=password
        self.__balance=balance

    def info(self):
        return f"AccountNumber={self._accountNumber}\tBalance={self.__balance}"



In [23]:
a=BankAccount(1,"p@ss",20000)
print(a.info())

AccountNumber=1	Balance=20000


In [24]:
print(a._accountNumber) # shouldn't try to access

# ore modify. But can do it
a._accountNumber =10

print(a.info())

1
AccountNumber=10	Balance=20000


#### modifying double underscored members

In [25]:
a.__balance=100000
print(a.__balance)

100000


### Is it really modified?

In [26]:
print(a.info())

AccountNumber=10	Balance=20000


### What is really happening?

* python internally changes the name of **double underscored prefixed members**
* it addes additional prefix of **\_ClassName\_**

```python
class BankAccount:
    def __init__(self, accountNumber, balance, password):
        self._accountNumber = accountNumber # saved as it is
        self.__balance=balance # saved as self._BankAcount__balance = balance

    def deposit(self, amount):
        self.__balance+=amount # self._BankAccount__balance+=amount
```

* This conversion takes place in all class methods
* This conversion doesn't take place when we access the value from outside

In [27]:
a= BankAccount(1,'p@ss',20000)
print(dir(a))

['_BankAccount__balance', '_BankAccount__password', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_accountNumber', 'info']


### if we try to modify from outside, a new field is added 

In [28]:
a.__balance=1000000

print(dir(a))

['_BankAccount__balance', '_BankAccount__password', '__balance', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_accountNumber', 'info']


### The original field (with modified names) doesn't change

In [29]:
print(a.info()) #internally uses original field with modified name

AccountNumber=1	Balance=20000


### Now that we know it, we can break it!

In [31]:
a=BankAccount(1,'p@ss',20000)

a._BankAccount__balance=100000 #MUST NEVER DO IT

print(a.info())

AccountNumber=1	Balance=100000
